In [121]:
import requests
import json
import time
from datetime import datetime

import sys
import pymysql

import pandas as pd
import numpy as np

In [93]:
def get_eqkmsg(npage, nrows=10):
        endpoint = 'http://apis.data.go.kr/1360000/EqkInfoService'
        mykey = 'hLgHjx4SRbAp3W6wWAgqvPqZB0dDxQlzAPhuLVz1nCYJ9WUW//16Hzz6vy/0n7rE4kB1kJ3/Ia7vxld3oid6IA=='
        datatype = 'JSON'
        params = {'ServiceKey':mykey, 'fromTmFc':eqkt, 'toTmFc':eqkt,
                'dataType':datatype, 'pageNo':npage, 'numOfRows':nrows}
        r = requests.get(endpoint+'/'+'getEqkMsg', params=params)
        res = json.loads(r.text).get('response')
        return res

In [131]:
def eqkmsg_count(fromt=None, tot=None, npage=1, nrow=10, gettoday=False):
    global eqkt, rescount, rpages, rrows

    if type(fromt) == int:
        fromt = str(fromt)
    if type(tot) == int:
        tot = str(tot)
    
    today = str(datetime.today())[0:10].split('-')
    eqkt = today[0]+today[1]+today[2]
    if gettoday:
        fromt, tot = eqkt, eqkt
        npage = str(int(npage))
        nrow = str(int(nrow))
    endpoint = 'http://apis.data.go.kr/1360000/EqkInfoService'
    mykey = 'hLgHjx4SRbAp3W6wWAgqvPqZB0dDxQlzAPhuLVz1nCYJ9WUW//16Hzz6vy/0n7rE4kB1kJ3/Ia7vxld3oid6IA=='
    datatype = 'JSON'

    if gettoday:
        print ('오늘의 지진 통보를 검색합니다.')
        params = {'ServiceKey':mykey, 'fromTmFc':eqkt, 'toTmFc':eqkt,
            'dataType':datatype, 'pageNo':npage, 'numOfRows':nrow}
    else:
        print ('조회기간: '+fromt[0:4]+'년'+fromt[4:6]+'월'+fromt[6:8]+'일 ~',tot[4:6]+'월'+tot[6:8]+'일')
        params = {'ServiceKey':mykey, 'fromTmFc':fromt, 'toTmFc':tot,
            'dataType':datatype, 'pageNo':npage, 'numOfRows':nrow}
    try:
        r = requests.get(endpoint+'/'+'getEqkMsg', params=params)
    except:
        print('기상청 api 호출에 오류 발생')
        sys.exit(1)

    if r.status_code == 200:
        try:
            res = json.loads(r.text).get('response')
            rcode = res.get('header')['resultCode']
            print('기상청 api 호출 성공 (API result code:',rcode+')')
        except:
            print('통보문 데이터를 불러오는데 실패했습니다.')
            sys.exit(1)

        if rcode == '00':
            rescount = res.get('body')['totalCount']
            rrows = int(res.get('body')['numOfRows'])
            rpages = (rescount//int(nrow)+1)
            if gettoday:
                print('오늘은 지진 통보가',rescount,'개 있습니다.')
            else:
                print('조회기간 동안 지진 통보는 총 ',rescount,'개 입니다.')
        elif rcode == '03':
            if gettoday:
                print('오늘은 지진 통보가 없습니다.')
            else:
                print('조회기간 동안 지진 통보가 없습니다.')

    if r.status_code != 200:
        print('기상청 api 호출 실패 code:',r.status_code)
        sys.exit(1)

    return rescount

In [125]:
todaycount = eqkmsg_count(gettoday=True)
todaycount

오늘의 지진 통보를 검색합니다.
기상청 api 호출 성공 (API result code: 03)
오늘은 지진 통보가 없습니다.


1

In [129]:
eqkcount = eqkmsg_count(20220412, 20220415)

조회기간: 2022년04월12일 ~ 04월15일
기상청 api 호출 성공 (API result code: 00)
조회기간동안 지진 통보는 총  3 개 입니다.


In [115]:
eqk_data()

[{'id': '20220414-00',
  'day': '2022041',
  'eqkt': '091500',
  'fct': '0940',
  'mt': 5.0,
  'loc': '일본 오키나와현 오키나와 서북서쪽 134km 해역',
  'lat': 26.8,
  'lon': 126.5,
  'mapURL': 'http://www.weather.go.kr/repositary/image/eqk/img/eqk_img_2_20220414091500.png',
  'fctype': 2}]

In [130]:
def eqk_data():
    global eqk_list
    eqk_dict = {}
    eqk_list = []
    for i in range(0, rpages):
        try:
            r = get_eqkmsg(str(i+1))
            raw = r.get('body').get('items')['item']
        except:
            print('지진 통보문 목록 불러오기 실패')
            break
        for j in range(0, len(raw)):
            eqk_dict = {}
            data = raw[j]
            eqk_dict['id'] = eqkt+'-'+str(i)+str(j)
            eqk_dict['day'] = str(data.get('tmEqk'))[0:7]
            eqk_dict['eqkt'] = str(data.get('tmEqk'))[8:14]
            eqk_dict['fct'] = str(data.get('tmFc'))[8:12]
            eqk_dict['mt'] = data.get('mt')
            eqk_dict['loc'] = data.get('loc')
            eqk_dict['lat'] = data.get('lat')
            eqk_dict['lon'] = data.get('lon')
            eqk_dict['mapURL'] = data.get('img')
            eqk_dict['fctype'] = data.get('fcTp')
            eqk_list.append(eqk_dict)
        time.sleep(1)

    pdcol = eqk_list[0].keys()
    eqk_df = pd.DataFrame(columns=pdcol, data=[eqk_list[i].values() for i in range(0,len(eqk_list))])
    return eqk_df

In [ ]:
def main():
    today = str(datetime.today())[0:10].split('-')
    eqkt = today[0]+today[1]+today[2]


In [6]:
if __name__ == '__main__':
    main()

기상청 api 호출 성공 (API result code: 03)
오늘의 지진 통보는 없습니다.


NameError: name 'rescount' is not defined

In [117]:
eqk_data()

지진 통보문 목록 불러오기 실패


IndexError: list index out of range

In [83]:
pdcol = eqk_list[0].keys()

In [72]:
eqk_data()

지진 통보문 목록 불러오기 실패
[]


In [84]:
pdcol = eqk_list[0].keys()
pd.DataFrame(columns=pdcol, data=[eqk_list[i].values() for i in range(0,len(eqk_list))])

,id,day,eqkt,fct,mt,loc,lat,lon,mapURL,fctype
0,20220414-00,2022041,091500,0940,5.0,일본 오키나와현 오키나와 서북서쪽 134km 해역,26.8,126.5,http://www.weather.go.kr/repositary/image/eqk/...,2


```mysql
CREATE DATABASE api1;
USE api1;
```

```mysql
CREATE TABLE dailyeqkinfo (id VARCHAR(255), day VARCHAR(255), eqkt INTEGER, fct INTEGER, mt DEMICAL(5,1), loc VARCHAR(255), lat DEMICAL(5,2), lon DEMICAL(5,2), mapURL VARCHAR(255), fctype VARCHAR(255), PRIMARY KEY(id)) ENGINE=InnoDB DEFAULT CHARSET='utf8';
```

```mysql
CREATE TABLE eqkfcinfo (fctype VARCHAR(255), fcname VARCHAR(255), PRIMARY KEY(fctype)) ENGINE=InnoDB DEFAULT CHARSET='utf8';
```

In [95]:
table = 'dailyeqkinfo'
col = ', '.join(eqk_list[0].keys())
place_holder = ', '.join(['%s']*len(eqk_list[0].keys()))
key_holder = ', '.join([i + '=%s' for i in eqk_list[0].keys()])
col

'id, day, eqkt, fct, mt, loc, lat, lon, mapURL, fctype'

In [134]:
'INSERT INTO {} ({}) VALUES ({}) ON DUPLICATE KEY UPDATE {}'.format(table, col, place_holder, key_holder)

'INSERT INTO dailyeqkinfo (id, day, eqkt, fct, mt, loc, lat, lon, mapURL, fctype) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s) ON DUPLICATE KEY UPDATE id=%s, day=%s, eqkt=%s, fct=%s, mt=%s, loc=%s, lat=%s, lon=%s, mapURL=%s, fctype=%s'

In [93]:
#place holder
', '.join(['%s']*len(eqk_list[0].keys()))

'%s, %s, %s, %s, %s, %s, %s, %s, %s, %s'

In [94]:
#key holder
[k + '=%s' for k in eqk_list[0].keys()]

['id=%s',
 'day=%s',
 'eqkt=%s',
 'fct=%s',
 'mt=%s',
 'loc=%s',
 'lat=%s',
 'lon=%s',
 'mapURL=%s',
 'fctype=%s']

In [ ]:
#DB 접속 시작
host = 'db-1.crtxwdze1fvo.ap-northeast-2.rds.amazonaws.com'
port = 3306
user = 'blackbird1227'
db = 'api1'
pw = 'gksqls1227!'
conn = pymysql.connect(host=host, user=user, passwd=pw, db=db, port=port, use_unicode=True, charset='utf8')
cursor = conn.cursor()

In [ ]:
#쿼리 실행
cursor.execute(que, list(eqk_list[0].values())*2)

In [98]:
cursor.fetchall()

dict_values(['20220413-0', '20220413', '145628', '1500', 2.4, '경북 울진군 남남동쪽 20km 해역', 36.82, 129.47, 'http://www.weather.go.kr/repositary/image/eqk/img/eqk_img_3_20220412145628.png', 3])

In [ ]:
#DB 접속 종료
conn.commit()
cursor.close()